In [ ]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def fetch_image_urls(query: str, max_links_to_fetch: int, wd: webdriver, sleep_between_interactions: int = 5):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img&tbs=isz:l"
    
    # load the page
    wd.get(search_url.format(q=query))
    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements(By.CSS_SELECTOR, "img.rg_i.Q4LuWd")
        number_results = len(thumbnail_results)

        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")

        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
                print('Link : ' ,wd.find_element(By.CSS_SELECTOR, 'img.n3VNCb.pT0Scc.KAlRDb').get_attribute('src') )
                thumbnail_results = wd.find_elements(By.CSS_SELECTOR, "img.rg_i.Q4LuWd")
                number_results = len(thumbnail_results)
            except Exception:
                continue

            # extract image urls
            actual_image = wd.find_element(By.CSS_SELECTOR, 'img.n3VNCb.pT0Scc.KAlRDb')
            if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))
                    print(image_urls)
          
            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
            else:
                print("Found:", len(image_urls), "image links, looking for more ...")
                time.sleep(1)
                continue
            
            load_more_button = wd.find_elements(By.CSS_SELECTOR,".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

            # move the result startpoint further down
            results_start = len(thumbnail_results)

    return image_urls


def download_image(folder_path:str,url:str, counter, search_term):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        f = open(os.path.join(folder_path, search_term.replace(' ', '_') + "_" + str(counter) + ".jpg"), 'wb')
        f.write(image_content)
        f.close()
        print(f"SUCCESS - saved {url} - as {folder_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

def search_and_download(search_term: str, target_path='./images', number_images=10):
    target_folder = os.path.join(target_path, '_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as wd: res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)

    counter = 0
    for elem in res:
        download_image(target_folder, elem, counter, search_term)
        counter += 1

        
# How to execute this code
# Step 1 : pip install selenium. pillow, requests
# Step 2 : pip install webdriver_manager
# Step 3 : make sure to install on your machine
# Step 5 : Your images will be saved to images folder


search_term = 'pickup'
# num of images to download
number_images = 10
search_and_download(search_term=search_term)